# Trénovanie hlbokých sietí - optimalizácia a rady - 2. časť

Na poslednom cvičení sme si prešli základné pravidlá návrhu modelov neurónových sietí, správny postup pri ich trénovaní a vyhodnocovaní, a pozreli sme sa na problém pretrénovania a ako ho dokážeme odhaliť. Témou dnešného cvičenia budú metódy, ktoré slúžia na optimalizáciu trénovania, aby ste vedeli model natrénovať rýchlejšie a s väčšou spoľahlivosťou.

## 1. Regularizácia

Cieľom regularizácie je predísť pretrénovaniu siete, t.j. chceme zabrániť tomu, aby ľubovoľný vstup, resp. ktorákoľvek váha ovplyvňovala výsledok vo vysokej, nadpriemernej miere. Pri použití regularizácie môžeme použiť aj väčšie siete bez rizika pretrénovania; čím väčšia je daná sieť, tým je menšia pravdepodobnosť, že regularizácia nám zvýši *bias*.

### 1.1. $L_{1}$ regularizácia

$L_{1}$ regularizácia pripočíta sumu absolútnych hodnôt váh do cieľovej funkcie, a tým zmenší koeficienty menej výrazných príznakov blízko 0. To má za následok, že niektoré príznaky sú ignorované, a sieť sa naučí riedšiu (*sparse*) reprezentáciu.

\begin{equation}
J(W, b) = \frac{1}{m} \sum_{i=1}^{m} L(\hat{y}^{2} - y^{2}) + \frac{\lambda}{2m} \sum_{l=1}^{L} \left \|w^{[l]}\right \|,
\end{equation}

kde $L$ je chybová funkcia, $\left \|w^{[l]}\right \|$ je suma absolútnych hodnôt všetkých váh a $\lambda$ je hyperparameter regularizácie.

### 1.2. $L_{2}$ regularizácia

$L_{2}$ regularizácia zabezpečuje, že hodnoty váh ostanú v istom intervale a tak sa sieť nebude nadmerne spoliehať na ani jednu z nich. Implementujeme ju rozšírením výpočty chyby siete:

\begin{equation}
J(W, b) = \frac{1}{m} \sum_{i=1}^{m} L(\hat{y}^{2} - y^{2}) + \frac{\lambda}{2m} \sum_{l=1}^{L} \left \|w^{[l]}\right \|_{F}^{2},
\end{equation}

kde $L$ je chybová funkcia, $\left \|w^{[l]}\right \|_{F}^{2}$ je tzv. Frobeniusova norma váh a $\lambda$ je hyperparameter regularizácie. Frobeniusova norma matice sa vypočíta nasledovne:

\begin{equation}
\left \|w^{[l]}\right \|_{F}^{2} = \sum_{i=1}^{n^{[l-1]}} \sum_{j=1}^{n^{[l]}}(w_{ij}^{[l]})^{2}
\end{equation}

pre *l*-tú vrstvu siete. Spôsob aktualizácia hodnoty váhy sa potom vykoná podľa vzorca:

\begin{equation}
\Delta w^{[l]} = \frac{\delta J}{\delta w^{[l]}} + \frac{\lambda}{m}w^{[l]}
\end{equation}

pre *l*-tú vrstvu siete.

Nevýhodou $L_{2}$ regularizácie je, že samotná $\lambda$ je ďalší hyperparameter, ktorého hodnotu vieme vhodne nastaviť len spôsobom pokus-omyl. Čím je $\lambda$ väčšia, tým viac sa budú hodnoty váh pohybovať okolo 0, tým pádom niektoré neuróny akokeby boli "vypnuté", vôbec ich sieť nebude používať - prakticky pracujeme s menšou sieťou. Ďalším dôsledkom použitia regularizácie je to, že suma vstupov neurónu bude takisto okolo 0, tým pádom sa použije iba lineárna časť aktivačných funkcií ako napr. sigmoidálna, a tým pádom nevieme pridať nelinearity do rozhodovania siete, teda zabránime pretrénovaniu.

$L_{2}$ regularizáciu v *PyTorchi* vieme použiť nastavením parametra `weight_decay` pre optimalizátor pri definícii trénovania (viď [dokumentácia napr. pre Adam](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html#torch.optim.Adam)).

### 1.3. Dropout

Dropout je alternatívny spôsob regularizácie, kde pre niektoré vrstvy (alebo každú vrstvu) v sieti určíme pravdepodobnosť toho, že náhodne vypneme niektoré neuróny - to znamená, že odstránime všetky vstupné a výstupné synapsie daného neurónu pre konkrétny výpočet, pracujeme teda s menšou sieťou. Efekt tejto metódy je podobný ako v prípade $L_{2}$ regularizácie - sieť sa naučí nespoliehať sa na žiadny vstup príliš, keďže nemá garantované, že daný vstup bude vždy k dispozícii. Čím je viac pravdepodobné, že sa sieť v danej vrstve pretrénuje, tým vyššia by mala byť pravdepodobnosť *dropoutu*. Pravdepodobnosť 0.0 znamená, že *dropout* nepoužijeme.

*Dropout* sa najčastejšie použije v počítačovom videní, ak nemáme k dispozícii veľa trénovacích dát. Nevýhoda *dropoutu* je to, že chyba siete $J$ nie je jasne definovaná, a práve preto jej hodnota nemusí klesať po každej iterácii trénovania. *Dropout* sa zvyčajne nepoužije vo vstupnej vrstve, ak áno, tak s hodnotou okolo 0.0.

V *PyTorchi* *dropout* použijeme pridaním špeciálnej `Dropout` vrstvy ([dokumentácia](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)).

### 1.4. Augmentácia dát

Spôsobom riešenia malého množstva trénovacích údajov je augmentácia datasetu, teda rozšírenie datasetu algoritmickými spôsobmi. V prípade obrázkov je to najčastejšie prevrátenie obrazu (horizontálne alebo vertikálne, podľa typu dát), náhodné ostrihanie obrazu (sústrediť sa na stred obrazu), pridanie skreslení, rotácia obrazu, atď. Treba si dávať pozor, aby tieto úpravy mali zmysel v kontexte datasetu a netreba zabudnúť, že augmentovaný dataset nikdy nebude dosahovať kvalitu väčšieho datasetu s viacerými príkladmi.

### 1.5. Skoré ukončenie trénovania

Ak chceme predísť pretrénovaniu siete, stopercentná metóda je ukončenie trénovania vo vhodnom momente. Preto pri každej iterácii trénovania zmeriame trénovaciu a validačnú chybu, a trénovanie ukončíme, ak trénovacia chyba ešte klesá, avšak validačná chyba začne rásť. Nevýhodou tejto metódy je to, že nevieme rozdeliť proces optimalizácie chyby a zabránenie pretrénovaniu.

## 2. Optimalizačné metódy

Cieľom optimalizačných metód je urýchliť konvergenciu siete ku globálnemu minimu. V niektorých prípadoch môžu takisto zabezpečiť, aby sieť vôbec konvergovala k tomuto minimu. Základné optimalizačné metódy sú normalizácia, vhodná inicializácia váh, mini-batch trénovanie a použitie optimalizátorov.

### 2. 1. Normalizácia vstupu

Normalizácia je základná metóda, ktorá **nikdy neuškodí trénovaniu** a práve preto sa používa veľmi často. Princíp za normalizáciou je skutočnosť, že neurónové siete fungujú najlepšie ak ich vstupy sú z istého intervalu (najčastejšie z rozmedzia 0-1). Z pohľadu chybového priestoru pomocou normalizácie dostaneme viac symetrický tvar chybovej hyperboly, vďaka čomu vieme použiť vyšší učiaci parameter (*learning rate*), vďaka čomu sieť bude konvergovať rýchlejšie a bez oscilácie.

Prvý základný spôsob normalizácie je odčítanie priemeru:

\begin{equation}
\mu=\frac{1}{m} \sum_{i=1}^{m}x^{(i)}
\end{equation}
\begin{equation}
x=x-\mu
\end{equation}

Druhá metóda je normalizácia variácie:

\begin{equation}
\sigma^2=\frac{1}{n} \sum_{i=1}^{n} x^{(i)2}
\end{equation}
\begin{equation}
x = x / \sigma^{2},
\end{equation}

kde musíme použiť rovnaké hodnoty $\mu$, resp. $\sigma^2$ pre validačnú a testovaciu množinu.

V *PyTorchi* normalizáciu vieme vykonať pomocou vrstvy `LayerNorm` ([dokumentácia](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)), keďže ale normalizáciu aplikujeme zvyčajne len pre vstup, môžeme tieto vstupy upraviť čisto štatistickými výpočtami.

Špeciálnym typom normalizácie je *batch* normalizácia, teda normalizácia v rámci *mini-batchu*. Vďaka nej je sieť robustnejšia, keďže je zabezpečaná, že priemer a variancia v rámci *mini-batchu* budú ustálené. Pri použití *batch* normalizácie nemusíme použiť *biasy* v predošlej vrstve, avšak mali by sme použiť trošku väčšie *mini-batche*. *PyTorch* ponúka samostatnú [vrstvu pre batch normalizáciu](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html#torch.nn.BatchNorm2d).

### 2.2. Inicializácia váh

Vhodnou inicializáciou váh vieme čiastočne zabrániť problémom *vanishing* a *exploding gradient*, ktoré spomaľujú trénovanie, keďže ak derivácia je príliš nízka alebo vysoká, sieť nedokáže nájsť minimum. Ak všetky váhy siete sú vyššie ako 1, aktivačné hodnoty budú stále vyššie (*explosion*, *exploding gradient*), ak všetky váhy majú hodnotu nižšiu ako 1, aktivácie budú stále nižšie (*implosion*, *vanishing gradient*).

Vhodnou inicializáciou teda chceme nastaviť čím väčší počet vstupov, tým menšiu hodnotu váh, pričom chceme zachovať istú variabilitu váh. Táto variabilita by mala byť okolo $\frac{1}{n}$ (pre ReLU $\frac{2}{n}$), kde $n$ je počet vstupov (počet neurónov v predošlej vrstve).

Vhodná inicializácia závisí od aktivačnej funkcie, pre *ReLU* sa najčastejšie používa inicializácia:

\begin{equation}
W^{[l]}=np.random.randn(shape) \cdot np.sqrt(\frac{2}{n^{[l-1]}})
\end{equation}

a pre *tanh* (Xavierova inicializácia):

\begin{equation}
W^{[l]}=np.random.randn(shape) \cdot np.sqrt(\frac{1}{n^{[l-1]}}),
\end{equation}

resp.

\begin{equation}
W^{[l]}=np.random.randn(shape) \cdot np.sqrt(\frac{2}{n^{[l-1]} + n^{[l]}})
\end{equation}

V *PyTorchi* nájdete niekoľko inicializátorov, [ich zoznam je dostupný tu](https://pytorch.org/docs/stable/nn.init.html).

### 2.3. Minibatch trénovanie

Ako sme už písali vyššie, vývoj hlbokých modelov je iteratívny proces, a práve preto chceme modely natrénovať a vyhodnotiť čo najrýchlejšie. Na druhej strane ale hlboké učenie funguje pri veľkom množstve údajov. Keby sme váhy aktualizovali vždy nad celým datasetom pomocou *backpropagationu*, váhy by sme aktualizovali len pomaly a nezískali by sme rýchlo prehľad o efektívnosti modelu pre riešenie daného problému. Práve preto zavedieme *mini-batche*, a váhy budeme aktualizovať na základe vypočítaných chýb nad príkladmi z tejto dávky údajov.

Ak máme dataset *m* príkladov a nastavíme veľkosť *mini-batchov* na *m*, dostaneme základný *batch gradient descent* (vždy konverguje, avšak potrebuje veľa času medzi dvoma aktualizáciami). Ak veľkosť *mini-batchov* je 1, hovoríme o *stochastic gradient descent* (smeruje k lokálnemu minimu, nikdy nekonverguje, strácame možnosť paralelizovať). Veľký rozdiel medzi *batch* a *mini-batch* trénovaním je to, že pri *batch* trénovaní chyba bude nižšia po každej iterácii. Pri *mini-batch* trénovaní to nemusí platiť, keďže výber príkladov do *mini-batchu* je náhodný, chyba by ale stále mala mať klesajúcu tendenciu.

Pri malých datasetoch ($m < 2000$) môžeme použiť *batch* trénovanie, pri väčších datasetoch by sme mali využiť *mini-batche* (zvyčajne veľkosť 64, 128, 256, 512 - dávajte si pozor, aby sa jeden *mini-batch* zmestil do pamäte CPU/GPU). V *PyTorchi* *mini-batche* vieme zadefinovať pomocou parametra `batch_size` v `DataLoaderi` ([dokumentácia](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)). Epocha potom reprezentuje prechádzanie celým datasetom.

## 3 Optimalizátory

Optimalizátory rozširujú základny *backprop*, resp. *gradient descent* algoritmy a upravujú spôsob aktualizácie váh. Sú založené na použití exponenciálnych kĺzavých priemerov, ktoré sú odolnejšie voči osciláciam a odstraňujú šum z číselného radu. Pre ľubovoľnú hodnotu $V$ ho vieme vypočítať podľa vzorca:

\begin{equation}
V_{i}=\beta V_{i-1} + (1-\beta)x_{i},
\end{equation}

pričom $V_{0}=0$. $\beta$ je hyperparameter, ktorý určuje, posledné približne koľko príkladov z číselného radu beriem do úvahy, t.j. posledných $\frac{1}{1-\beta}$ príkladov. Napríklad, ak $\beta=0.9$, kĺzavý priemer bude vypočítaný na základe posledných desiatich hodnôt.

V niektorých prípadoch sa používa aj korekcia *bias*u, a to nasledovne:

\begin{equation}
V_{i}=\frac{\beta V_{i-1} + (1- \beta)x_{i}}{1-\beta^t}
\end{equation}

pre *t*-ú epochu, resp. *mini-batch*.

### 3.1. Gradient Descent with Momentum

*Gradient Descent with Momentum* je rozšírením základného algoritmu spätného šírenia chyby, v ktorom sa nepoužije samotný vypočítaný gradient $\frac{\delta J}{\delta w}$, ale exponenciálny kĺzavý priemer gradientu. Takto je oscilácia pri trénovaní menšia, korekcia *biasu* zvyčajne nie je potrebná. Algoritmus prebieha nasledovne:

1. výpočet gradientu $\Delta W$ na danom *mini-batche*
2. výpočet kĺzavého priemeru

\begin{equation}
V_{\Delta W} = \beta V_{\Delta W} + (1 - \beta)\Delta W
\end{equation}

3. aktualizácia váh

\begin{equation}
W=W- \alpha V_{\Delta W}
\end{equation}

Najčastejšie je hodnota $\beta$ nastavená na 0.9, a vektor $V_{\Delta W}$ je inicializovaný ako matica núl.

### 3.2. Root Mean Square prop (RMSprop)

*RMSprop* používa trošku iný spôsob výpočtu kĺzavého priemeru použitím umocnenia matice $\Delta W$. Takto sa zabezpečuje, že pri väčšej zmene chyby podľa istej váhy bude zmena váhy menšia - malá oscilácia. Optimalizátor RMSprop umožňuje použitie vyššej hodnoty učiaceho parametra.

1. výpočet gradientu $\Delta W$ na danom *mini-batche*
2. výpočet kĺzavého priemeru

\begin{equation}
S_{\Delta W} = \beta_{2} S_{\Delta W} + (1 - \beta_{2})\Delta W^2
\end{equation}

3. aktualizácia váh

\begin{equation}
W=W-\alpha \frac{\Delta W}{\sqrt{S_{\Delta W}} + \epsilon},
\end{equation}

kde $\epsilon$ je malé číslo (okolo $10^{-8}$).

### 3.3. Adaptive moment estimation (Adam)

Optimalizátor Adam je kombinácia predošlých dvoch metód:

1. výpočet gradientu $\Delta W$ na danom *mini-batche*
2. výpočet kĺzavého priemeru

\begin{equation}
V_{\Delta W} = \beta_1 V_{\Delta W} + (1 - \beta_1)\Delta W
\end{equation}
\begin{equation}
S_{\Delta W} = \beta_{2} S_{\Delta W} + (1 - \beta_{2})\Delta W^2
\end{equation}

3. korekcia biasu

\begin{equation}
V^{corr}_{\Delta W}=\frac{V_{\Delta W}}{1-\beta_1^t}
\end{equation}
\begin{equation}
S^{corr}_{\Delta W}=\frac{S_{\Delta W}}{1-\beta_2^t}
\end{equation}

4. aktualizácia váh

\begin{equation}
W=W-\alpha \frac{V_{\Delta w}^{corr}}{\sqrt{S_{\Delta W}^{corr}}+\epsilon}
\end{equation}

Optimalizátor Adam má niekoľko hyperparametrov:

* $\alpha$ - učiaci parameter, je potrebné ho doladiť
* $\beta_1 = 0.9$ zvyčajne
* $\beta_2 = 0.999$ zvyčajne
* $\epsilon = 10^{-8}$ zvyčajne

### 3.4. Learning Rate Decay

Pri trénovaní by sme ideálne chceli postupne znižovať hodnotu učiaceho parametra ak sa blížime k minimu. Takto vieme zabezpečiť konvergenciu siete, čo nie je garantované pri použití *mini-batchov*. Na znižovanie hodnoty existuje niekoľko metód:

1. $\alpha=\frac{1}{1+decay \cdot epoch} \cdot \alpha_0$
2. $\alpha=0.95^{epoch} \cdot \alpha_0$ - exponenciálny
3. $\alpha = \frac{k}{\sqrt{epoch}} \cdot \alpha_0$ alebo $\alpha = \frac{k}{\sqrt{t}} \cdot \alpha_0$ - kde $k$ je ďalší hyperparameter a konštanta, a $t$ je číslo *mini-batchu*
4. diskrétne znižovanie - hodnotu zmenšíme na polovicu po epoche alebo *mini-batchi*
5. manuálne znižovanie

## 4. Ladenie hyperparametrov

Ako vidíte, pri návrhu hlbokých sietí a pri ich trénovaní použijeme niekoľko hyperparametrov, ako napríklad: učiaci parameter, parametre optimalizátora, počet vrstiev, počet neurónov, *learning rate decay*, veľkosť *mini-batchu*. V poradí podľa dôležitosti (do akej miery ovplyvňujú presnosť modelu alebo rýchlosť trénovania):

1. učiaci parameter ($\alpha$)
2. veľkosť *mini-batchu*, počet neurónov, parametre optimalizátora
3. počet vrstiev, *learning rate decay*.

Otázka je, ako nájdeme vhodnú kombináciu hodnôt týchto parametrov pre efektívne trénovanie. Keďže neexistujú žiadne heuristiky a rady, ktoré by fungovali všeobecne, najjednoduchšia metóda je vyskúšať náhodné hodnoty parametrov, a porovnanie (krosvalidácia) presností modelov s danými parametrami. Týmto spôsobom jednoducho nájdeme, ktoré parametre ovplyvňujú úspech modelu najviac, a vieme sa sústrediť na ne. V prípade diskrétnych parametrov by sme si mali zvoliť vhodnú stupnicu možných hodnôt a z nej vybrať náhodné hodnoty - najčastejšie logaritmická. V Pythone viete náhodný výber z logaritmickej stupnice implementovať nasledovne (napr. pre interval $[10^{-4}, 10^0]$):

In [ ]:
r = -4 * np.random.ran()  # r in [-4, 0]
alpha = 10 ** r

Túto metódu môžete použiť pre učiaci parameter a parametre kĺzavého priemeru (optimalizátora - výber pre $1 - \beta$).

Ak nájdete oblasť v ktorom sa modely trénujú presnejšie, mali by ste sa sústrediť na ňu, a znova urobiť niekoľko iterácií náhodných výberov.

Ďalším dobrým krokom je hľadať vedecké články o podobných úlohách aj z iných domén, možno nájdete aplikovateľné riešenie. Vo fáze života modelu je treba niekoľkokrát preskúšať nastavenie hyperparametrov, možno dataset alebo počítačový systém sa zmenil.

## Použitá literatúra

* [Andrew Ng: Improving Deep Neural Networks - Hyperparameter Tuning](https://www.youtube.com/watch?v=1waHlpKiNyY&list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc)